In [38]:
from os.path import join
from datetime import time, datetime
import os
import glob
import numpy as np
from skimage.draw import polygon
import SimpleITK as sitk
#import pydicom as dicom
#from pydicom.tag import Tag
import sys
import csv
import pandas as pd
import shutil

In [39]:
import yaml
with open('config_CheckNiiOk.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
    
    itvtot = config['itv_tot_labels']
    itvtum = config['itv_tumor_labels']
    itvnod = config['itv_ln_labels']

    
    gtvtot = config['gtv_tot_labels']
    gtvtum = config['gtv_tumor_labels']
    gtvnod = config['gtv_ln_labels']
    gtv00 = config['gtv_expi00_labels']

    rootp_Nii = config['root_Nii']

    aveTags = config['aveTags']
    bpTags =  config['bPTags']

    savecsv_path = config['save_csv']
    fieldnames = config['fieldnames']

PxList = os.listdir(rootp_Nii)
print(PxList)
target_labels = itvtot + itvtum+itvnod+gtvtot+gtvtum+gtvnod

['4025031', '2156167', '2154195', '3932820', '619086', '3007430', '147625', '7786608', '8728753', '1217167', '8967750', '2131592', '7676177', '810589', '2894544', '7513114', '4664141', '766115', '3307795', '9588899', '23840', '2324157', '2123820', '5443347', '4023673', '2856145', '5317512', '5635761', '3741290', '1034109', '1353431', '5497325', '14504', '170672', '6465961', '2628217', '2788596', '6841834', '7233645', '2370962', '367466', '234628', '5686032', '2186015', '9593888', '9222012', '8168463', '3853415', '3464709', '2758081', '9936840', '9193538', '561128', '2277478', '5960268', '3558287', '654137', '13194', '2326539', '5468590', '3059294', '9212857', '4572900', '110867', '1136033', '7838986', '4965499', '2049190', '2196145', '2168497', '7626090', '5539952', '3157977', '7176049', '2334091', '3409329', '2256800', '2209191', '9598757', '2132029', '8248305', '1822090', '9854449', '9646447', '5881434', '7313332', '2296266', '986614', '1439985', '5612712', '6577371', '1302698', '667

In [40]:
gtv00

['GTVtumor_insp']

In [46]:
patientListComplete = []
missingDelineation = 0
missingPlanCT =0
missingBP =0
corruptedPx_dict = {}
gtv00_listPx = []


with open(savecsv_path, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for patientID in PxList:
        itvtot_b,itvtum_b,itvnod_b=(False,False,False)
        gtvtot_b,gtvtum_b,gtvnod_b=(False,False,False)
        BP0_b,BP10_b,BP20_b,BP30_b,BP40_b,BP50_b,BP60_b,BP70_b,BP80_b,BP90_b,BP100_b=(False,False,False,False,False,False,False,False,False,False,False)
        planCT_b = False
        check_b = False

        for root, dirs, files in os.walk(os.path.join(rootp_Nii,patientID), topdown=True):
            for f in files: 
                #Average
                if any(planct_tt in f.lower() for planct_tt in aveTags):
                    planCT_b = True
                #ITV
                if  any(itvtot_tt in f for itvtot_tt in itvtot):
                    itvtot_b = True
                if any(itvtum_tt in f for itvtum_tt in itvtum):
                    itvtum_b = True
                if any(itvnod_tt in f for itvnod_tt in itvnod):
                    itvnod_b = True
                #GTV
                if any(gtvtot_tt in f for gtvtot_tt in gtvtot):
                    gtvtot_b = True
                if any(gtvtum_tt in f for gtvtum_tt in gtvtum):
                    gtvtum_b = True
                if any(gtvnod_tt in f for gtvnod_tt in gtvnod):
                    gtvnod_b = True
                if any(gtv00_tt in f for gtv00_tt in gtv00):
                    gtv00_listPx.append(patientID)
                #BP
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["0%"]):
                    BP0_b =True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["10%"]):
                    BP10_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["20%"]):
                    BP20_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["30%"]):
                    BP30_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["40%"]):
                    BP40_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["50%"]):
                    BP50_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["60%"]):
                    BP60_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["70%"]):
                    BP70_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["80%"]):
                    BP80_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["90%"]):
                    BP90_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["100%"]):
                    BP100_b = True
    
        if planCT_b and (itvtot_b or itvtum_b or itvnod_b) and (gtvtot_b or gtvtum_b or gtvnod_b) and (BP50_b or BP60_b) and (BP0_b,BP100_b,BP90_b):
            check_b = True
            patientListComplete.append(patientID)
        if not((itvtot_b or itvtum_b or itvnod_b) and (gtvtot_b or gtvtum_b or gtvnod_b)):
            missingDelineation +=1
            corruptedPx_dict[patientID] = "MissingDelineations"
            print(files)
        if not((BP50_b or BP60_b) and (BP0_b,BP100_b,BP10_b,BP90_b)):
            missingBP +=1
            corruptedPx_dict[patientID] = "Missing Peak Phases"
        if not(planCT_b):
            missingPlanCT +=1
            corruptedPx_dict[patientID] = "Missing Plan CT"
        writer.writerow({
            "PatientID": patientID,"PlanCT": planCT_b,
            "ITV_Total": itvtot_b,"ITV_Tumor": itvtum_b,"ITV_Nodal": itvnod_b,
            "GTV_Total": gtvtot_b,"GTV_Tumor": gtvtum_b,"GTV_Nodal": gtvnod_b,
            "BP_0%": BP0_b,"BP_10%": BP10_b,"BP_90%": BP90_b,"BP_100%": BP100_b,
            "BP_20%": BP20_b,"BP_30%": BP30_b,
            "BP_40%": BP40_b,"BP_50%": BP50_b,"BP_60%": BP60_b,
            "BP_70%": BP70_b,"BP_80%": BP80_b,"Check":check_b
        })


['100_t0pr97_-_4ar-5_-_49.nii.gz', '101_t10pr6_-_15ar42_-_70.nii.gz', '102_t20pr17_-_25ar62_-_82.nii.gz', '103_t30pr25_-_33ar72_-_86.nii.gz', '104_t40pr36_-_44ar73_-_88.nii.gz', '105_t50pr46_-_54ar73_-_89.nii.gz', '106_t60pr56_-_65ar74_-_89.nii.gz', '107_t70pr65_-_74ar44_-_90.nii.gz', '108_t80pr76_-_85ar25_-_87.nii.gz', '109_t90pr84_-_94ar10_-_60.nii.gz', '111_ave-ip10_0_10_20_30_40_50_60_70_80_90.nii.gz']
['10_resplow__20__b30s__100_ex.nii.gz', '11_resplow__20__b30s__60_ex.nii.gz', '12_resplow__20__b30s__30_ex.nii.gz', '3_thorax_2mm__standaard__b40f.nii.gz', '7_resplow__20__b30s__0_in.nii.gz', '8_resplow__20__b30s__40_in.nii.gz', '9_resplow__20__b30s__90_in.nii.gz']
['12_thorax_2mm_met_toploodje.nii.gz', '3_resplow__20__b30s__0_in.nii.gz', '4_resplow__20__b30s__40_in.nii.gz', '5_resplow__20__b30s__90_in.nii.gz', '6_resplow__20__b30s__100_ex.nii.gz', '7_resplow__20__b30s__60_ex.nii.gz', '8_resplow__20__b30s__30_ex.nii.gz']
['100_t0pr98_-_4ar10_-_29.nii.gz', '101_t10pr8_-_15ar32_-_59.ni

In [45]:
print("Complete patients",len(patientListComplete),"Len",len(PxList))
print("Missing Delineation",missingDelineation,"BP",missingBP,"PlanCT",missingPlanCT)
print("Len Insp GTV",len(gtv00_listPx))

Complete patients 621 Len 737
Missing Delineation 24 BP 91 PlanCT 2
Len Insp GTV 17


In [43]:
len(corruptedPx_dict)

116

In [44]:
corruptedPx_dict

{'7513114': 'Missing Peak Phases',
 '766115': 'Missing Peak Phases',
 '23840': 'Missing Peak Phases',
 '4023673': 'Missing Peak Phases',
 '2856145': 'Missing Peak Phases',
 '5317512': 'MissingDelineations',
 '5635761': 'Missing Peak Phases',
 '3741290': 'Missing Peak Phases',
 '367466': 'Missing Peak Phases',
 '3853415': 'Missing Peak Phases',
 '2758081': 'Missing Peak Phases',
 '9193538': 'Missing Peak Phases',
 '654137': 'Missing Peak Phases',
 '13194': 'MissingDelineations',
 '5468590': 'Missing Peak Phases',
 '1136033': 'MissingDelineations',
 '4965499': 'MissingDelineations',
 '2334091': 'MissingDelineations',
 '9854449': 'Missing Peak Phases',
 '9646447': 'Missing Peak Phases',
 '8269736': 'Missing Peak Phases',
 '9600274': 'Missing Peak Phases',
 '346733': 'Missing Peak Phases',
 '2345065': 'Missing Peak Phases',
 '9915436': 'Missing Peak Phases',
 '3058377': 'MissingDelineations',
 '6934596': 'MissingDelineations',
 '4239862': 'MissingDelineations',
 '3740147': 'Missing Peak Ph